<a href="https://colab.research.google.com/github/Pinini777/52079/blob/main/SemanticSearch_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%capture evita que la salida ensucie la pantalla, solo muestra errores si los hay.
%%capture
!pip install sentence-transformers torch numpy

Usaremos sentence-transformers al principio porque es un wrapper **"Pythonic"** excelente sobre Hugging Face, ideal para entender el concepto de Embedding sin pelear con tokenizers manuales todavía.

In [6]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Any, Union

class SemanticEngine:
    """
    Motor de búsqueda semántica basado en Embeddings vectoriales.
    Utiliza modelos Transformer para convertir texto a vectores en R^n.
    """

    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        """
        Inicializa el motor.

        Args:
            model_name (str): ID del modelo en Hugging Face.
                              'all-MiniLM-L6-v2' es ligero y eficiente (384 dimensiones).
        """
        # Verificación de Hardware: Esto es CRÍTICO en producción.
        # Si hay GPU, la usamos. Si no, CPU.
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"🚀 Iniciando Engine en dispositivo: {self.device}")

        # Carga del modelo
        # Almacenamos el modelo como atributo privado (encapsulamiento)
        try:
            self._model = SentenceTransformer(model_name, device=self.device)
        except Exception as e:
            raise RuntimeError(f"Error al cargar el modelo {model_name}: {e}")

        # Aquí guardaremos nuestra "Base de Datos" vectorial temporalmente (en RAM)
        # En el futuro, esto se reemplaza por una Vector DB (ej: ChromaDB o Pinecone)
        self._knowledge_base: List[Dict[str, Any]] = []
        self._embeddings_matrix: Union[np.ndarray, None] = None

    def add_documents(self, documents: List[str]) -> None:
        """
        Recibe una lista de textos (strings), los vectoriza y almacena.
        """
        # PASO 1: Guardar texto crudo (Metadata)
        # Iteramos sobre la lista de documentos y guardamos cada uno en un diccionario.
        # Usamos diccionarios {'text': doc} por si a futuro queremos agregar 'fecha', 'autor', etc.
        for doc in documents:
            self._knowledge_base.append({"text": doc})

        # PASO 2: Generar Embeddings (La Magia Matemática)
        # No hacemos un bucle for aquí. Pasamos la lista ENTERA.
        # La librería optimiza esto en lotes (batches) para la CPU/GPU.
        # convert_to_numpy=True nos asegura que devuelve un Array (matriz), no una lista.
        new_embeddings = self._model.encode(documents, convert_to_numpy=True)

        # PASO 3: Gestión de la Matriz en Memoria
        if self._embeddings_matrix is None:
            # Si es la primera vez, la matriz es igual a los nuevos embeddings
            self._embeddings_matrix = new_embeddings
        else:
            # Si ya teníamos datos, "concatenamos" la nueva matriz debajo de la vieja.
            # axis=0 significa "agregar filas" (verticalmente).
            self._embeddings_matrix = np.vstack((self._embeddings_matrix, new_embeddings))

        # Feedback para el desarrollador
        print(f"✅ Se procesaron {len(documents)} documentos.")
        print(f"📊 Dimensión de la Base de Datos Vectorial: {self._embeddings_matrix.shape}")

    def _cosine_similarity(self, vec_a: np.ndarray, vec_b: np.ndarray) -> float:
        """
        Calcula la similitud coseno entre dos vectores.
        """
        #1. Producto Punto(Dot Product)
        #En Algebra: A . B
        #En Numpy: np.dot(A,B)
        dot_product = np.dot(vec_a, vec_b)
        #2. Norma(Magnitud del vector)
        # En Algebra: ||A||
        #En Numpy: np.linalg.norm(A)
        norm_a = np.linalg.norm(vec_a)
        norm_b = np.linalg.norm(vec_b)

        if norm_a == 0 or norm_b == 0:
          return 0.0

        return dot_product / (norm_a * norm_b)

    def search(self, query: str, top_k: int = 3) -> List[str]:
        """
        Busca los documentos más similares a la query.
        """
        # 1. Vectorizar la consulta (Query Embedding)
        # El modelo convierte el texto de busqueda al mismo espacio vectorial
        query_vector = self._model.encode([query], convert_to_numpy= True)[0]

        # 2. Calcular similitudes
        # Aca se guardan tuplas: (indice, puntaje_similitud)
        similarities = []
        #Iteramos sobre nuestra matriz de vectores guardados
        for idx, doc_vector in enumerate(self._embeddings_matrix):
            score = self._cosine_similarity(query_vector, doc_vector)
            similarities.append((idx, score))

        # 3. Ordenar resultados (ranking)
        # Sorted ordena listas, key=lambda x: x[1] le dice "ordena basandote en el puntaje"
        # reverse=Truepara que el mayor puntaje(mas similar) quede primero.
        similarities = sorted(similarities, key = lambda x: x[1], reverse=True)
        # 4. Recuperar los textos originales
        results = []
        # CORRECCIÓN 1: Asegúrate de que las variables del bucle coincidan (idx, score)
        for idx, score in similarities[:top_k]:
            # CORRECCIÓN 2: _knowledge_base todo en minúsculas
            original_text = self._knowledge_base[idx]['text']
            results.append(f"Similitud: {score:.4f} | Texto: {original_text}")

        return results

In [7]:
# 1. Instanciamos nuestro motor (Esto descargará el modelo la primera vez)
engine = SemanticEngine()

# 2. Le damos "conocimiento" (Documentos desordenados)
textos_prueba = [
    "El perro es el mejor amigo del hombre",  # Concepto: Mascota/Perro
    "La inteligencia artificial está revolucionando el mundo", # Concepto: Tecnología
    "Me encanta comer pizza con mucho queso", # Concepto: Comida
    "Los gatos son animales muy independientes", # Concepto: Mascota/Gato
    "Python es un lenguaje de programación excelente", # Concepto: Tecnología/Coding
]

engine.add_documents(textos_prueba)

# 3. Hacemos búsquedas SEMÁNTICAS (Nota que no uso las palabras exactas)
print("\n--- PRUEBA 1: Buscando 'animal que ladra' ---")
# La palabra "ladra" NO está en los textos, pero el modelo sabe que perro y ladrar están relacionados.
resultados = engine.search("animal que ladra", top_k=2)
for r in resultados: print(r)

print("\n--- PRUEBA 2: Buscando 'algo de informática' ---")
resultados = engine.search("algo de informática", top_k=2)
for r in resultados: print(r)

🚀 Iniciando Engine en dispositivo: cpu
✅ Se procesaron 5 documentos.
📊 Dimensión de la Base de Datos Vectorial: (5, 384)

--- PRUEBA 1: Buscando 'animal que ladra' ---
Similitud: 0.5776 | Texto: Los gatos son animales muy independientes
Similitud: 0.3675 | Texto: Me encanta comer pizza con mucho queso

--- PRUEBA 2: Buscando 'algo de informática' ---
Similitud: 0.4483 | Texto: La inteligencia artificial está revolucionando el mundo
Similitud: 0.4340 | Texto: El perro es el mejor amigo del hombre


=============================================================================
RESUMEN TÉCNICO: MOTOR DE BÚSQUEDA SEMÁNTICA (PROYECTO ALPHA)
=============================================================================

1. ARQUITECTURA:
   - Implementación de un sistema de "Dense Retrieval" (Recuperación Densa).
   - Patrón de diseño orientado a objetos (Clase SemanticEngine) con principios
     de encapsulamiento para el modelo y la base de conocimiento.

2. MATEMÁTICA APLICADA:
   - Transformación de texto no estructurado a Vectores en R^384 (Embeddings).
   - Cálculo de similitud mediante Similitud del Coseno (Cosine Similarity).
   - Uso de Álgebra Lineal (Producto Punto y Normas L2) sobre matrices NumPy
     para ranking de relevancia.

3. STACK TECNOLÓGICO:
   - Python 3.10+ (Type Hinting).
   - PyTorch (Backend de tensores).
   - Sentence-Transformers (Modelo: all-MiniLM-L6-v2).
   - NumPy (Operaciones vectoriales de alto rendimiento).

4. RESULTADO:
   - Capacidad de "Semantic Search": El motor recupera documentos basados en
     el significado conceptual y no solo en coincidencia exacta de palabras clave.